In [1]:
import numpy as np
import csv
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold
from scipy import interp
from  matplotlib import pyplot as plt
from sklearn import cross_validation
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn import metrics, preprocessing
import pickle
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [2]:
df = pd.read_json("~/Documents/AML/HW2/train.json")

In [3]:
df.head(5)

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [4]:
ingredients_list = df['ingredients'].as_matrix()

In [5]:
def clean_data(matrix):
    new_list = []
    for array in matrix:
        row = []
        for ingr in array:
            if ' ' in ingr:
#                 print(ingr)
                new_ingr_split = ingr.split()
#                 print(new_ingr_split)
                for item in new_ingr_split:
                    if item != 'the' and ingr != 'of' and ingr != 'and' and ingr != 'a':
                        row.append(item.lower())
            else:
                row.append(ingr.lower())
        new_list.append(row)
    return new_list

In [6]:
new_matrix = clean_data(ingredients_list)

In [7]:
len(new_matrix)
print(len(new_matrix), new_matrix[1:10])

39774 [['plain', 'flour', 'ground', 'pepper', 'salt', 'tomatoes', 'ground', 'black', 'pepper', 'thyme', 'eggs', 'green', 'tomatoes', 'yellow', 'corn', 'meal', 'milk', 'vegetable', 'oil'], ['eggs', 'pepper', 'salt', 'mayonaise', 'cooking', 'oil', 'green', 'chilies', 'grilled', 'chicken', 'breasts', 'garlic', 'powder', 'yellow', 'onion', 'soy', 'sauce', 'butter', 'chicken', 'livers'], ['water', 'vegetable', 'oil', 'wheat', 'salt'], ['black', 'pepper', 'shallots', 'cornflour', 'cayenne', 'pepper', 'onions', 'garlic', 'paste', 'milk', 'butter', 'salt', 'lemon', 'juice', 'water', 'chili', 'powder', 'passata', 'oil', 'ground', 'cumin', 'boneless', 'chicken', 'skinless', 'thigh', 'garam', 'masala', 'double', 'cream', 'natural', 'yogurt', 'bay', 'leaf'], ['plain', 'flour', 'sugar', 'butter', 'eggs', 'fresh', 'ginger', 'root', 'salt', 'ground', 'cinnamon', 'milk', 'vanilla', 'extract', 'ground', 'ginger', 'powdered', 'sugar', 'baking', 'powder'], ['olive', 'oil', 'salt', 'medium', 'shrimp', 'pe

In [8]:
ingr_Dict = dict()
for item in new_matrix:
    for ingredient in item:
        if ingredient in ingr_Dict:
            ingr_Dict[ingredient] +=1
        else:
            ingr_Dict[ingredient] =1

In [9]:
len(ingr_Dict)

3186

In [10]:
consolidated_list = list(ingr_Dict.keys())

In [11]:
print (consolidated_list[0:2])

['sandwiches', 'flanken']


In [12]:
factorize_dict = dict()
k=0
for item in new_matrix:
    for ingredient in item:
        if ingredient not in factorize_dict:
            k+=1
            factorize_dict[ingredient] =k

In [13]:
len(factorize_dict)

3186

In [14]:
factorized_list = []
for array in new_matrix:
    row = []
    for x in array:
        num = factorize_dict[x]
        row.append(num)
    factorized_list.append(row)

In [15]:
print(factorized_list[0:2])

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [17, 18, 19, 8, 20, 6, 19, 3, 8, 21, 22, 23, 6, 24, 25, 26, 27, 28, 29]]


In [16]:
factorized_consolidated_list = list(factorize_dict.values())

In [17]:
factorized_consolidated_list.sort()

In [21]:
print(len(factorized_consolidated_list))

3186


In [25]:
blank_data = np.empty((len(factorized_list), len(factorized_consolidated_list)+1))

In [26]:
len(factorized_list[0])

16

In [27]:
s = 0
while s < len(factorized_list):
    for item in factorized_list[s]:
        blank_data[s][item] = 1
    s+=1
#     print("s ", s)

In [28]:
print(blank_data[-1])

[ 0.  0.  0. ...,  0.  0.  0.]


In [29]:
df['cuisine'] = pd.factorize(df['cuisine'])[0]

In [31]:
Y = df.iloc[:,0].values

In [32]:
X= blank_data

In [33]:
print ("X shape is: ", X.shape, "Y shape is" , Y.shape, )

X shape is:  (39774, 3187) Y shape is (39774,)


In [34]:
##make sure there are no nan values
np.any(np.isnan(X)), np.any(np.isnan(Y))

(False, False)

In [35]:
sum = 0
for item in X:
    for num in item: 
        sum = sum + num

In [36]:
print(sum)

752205.0


In [37]:
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(
    X, Y, test_size=0.25, random_state=42)

In [49]:
##uses K fold to gather scores for each fold
def evaluate(fold):
    # Input: A tuple of (train, test) indices
    train, test = fold
    return LogisticRegression().fit(X[train], Y[train]) \
        .score(X[test], Y[test])

all_folds = cross_validation.KFold(len(X), n_folds=3)

In [51]:
## mean scores are 
scores = []
for fold in all_folds:
    scores.append(evaluate(fold))


In [55]:
np.mean(scores)

0.78229496656107012

In [72]:
scores = cross_validation.cross_val_score(clf, X, Y, cv=3)
scores.mean()

0.78199308839376502

In [38]:
clf = LogisticRegression(C=1)
clf.fit((X_train),Y_train)
clf.score((X_test),Y_test)

0.78197908286403861

In [73]:
scores_gnb = cross_validation.cross_val_score(gnb, X, Y, cv=3)
scores_gnb.mean()

0.26502153877819484

In [40]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, Y_train).predict(X_test)

In [57]:
print("accuracy is: " ,(Y_test == y_pred).sum()/ len(Y_test))

accuracy is:  0.255028157683


In [42]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
y_pred_bnb = bnb.fit(X_train, Y_train).predict(X_test)

Number of mislabeled points out of a total %d points : %d (array([ 0.,  0.,  0., ...,  0.,  0.,  0.]), 0.74497184231697511)


In [58]:
print("accuracy is: " ,(Y_test == y_pred_bnb).sum()/ len(Y_test))

accuracy is:  0.71621078037


In [74]:
scores_gnb = cross_validation.cross_val_score(gnb, X, Y, cv=3)
scores_bnb.mean()

NameError: name 'scores_bnb' is not defined